## Import Elevations

In [ ]:
import pandas as pd
import requests
import json

In [ ]:
#converts georgia lats and longs to values in signed decimal notation
def convertlatlong(loc,lat=True):
    deg = float(str(loc)[0:2])
    minute = float(str(loc)[2:4])
    sec = float(str(loc)[4:5]+'.'+str(loc)[5:])
    final = round(deg+minute/60+sec/3600,3)
    if lat == False:
        final = -final
    return final

#Get elevation with one request at a time, due to Open Elevation API limitations in retrieving elevation data along a profile (easy to exceed max distance between points)
def geteachelevation(df):
    #Initialize variables
    location_df = df[['LAT_016','LONG_017']]
    location_df.insert(loc=2,column='lat',value=0)
    location_df.insert(loc=3,column='long',value=0)
    location_df.insert(loc=4,column='elevation',value=0)
    API_KEY = 'lGkweI18DIdVVlSAQsnLKi4BGAhH8YNf'
    
    #parse location_df to convert lat/long to standard format
    location_df['lat'] = location_df['LAT_016'].apply(lambda x: convertlatlong(x,lat=True))
    location_df['long'] = location_df['LONG_017'].apply(lambda x: convertlatlong(x,lat=False))
    location_df.reset_index(drop=True,inplace=True)
    
    #Request elevations
    for index, row in location_df.iterrows():
        api_string = str(row[2])+','+str(row[3])       
        #print(api_string)
        response = requests.get('http://open.mapquestapi.com/elevation/v1/profile?key='+API_KEY+'&shapeFormat=raw&unit=f&latLngCollection='+api_string)
        #parse json for elevations
        elev = json.loads(response.text)
        #print(elev)
        #print(y)
        if 'elevationProfile' in elev.keys():
            if len(elev['elevationProfile']) == 1:
                elevation = elev['elevationProfile'][0]['height']
            else:
                elevation = -32768
        else:
            elevation = -32768
        location_df.iloc[index,4] = elevation
        
    return location_df

#Uses Mapquest Open Elevation API to get elevations corresponding to specific latitude/longitude pairs, 5 at a time
def getelevation(df):
    #Initialize variables
    max_query_length = 5
    location_df = df[['LAT_016','LONG_017']]
    location_df.insert(loc=2,column='lat',value=0)
    location_df.insert(loc=3,column='long',value=0)
    location_df.insert(loc=4,column='elevation',value=0)
    length = location_df.shape[0]
    API_KEY = 'UKwrmeHVQ8q3ukLXjaBs74NZ822PF7DQ'
    
    #parse location_df to convert lat/long to standard format
    location_df['lat'] = location_df['LAT_016'].apply(lambda x: convertlatlong(x,lat=True))
    location_df['long'] = location_df['LONG_017'].apply(lambda x: convertlatlong(x,lat=False))
    location_df.sort_values(by=['lat','long'], inplace=True)
    location_df.reset_index(drop=True, inplace=True)
    #Request elevations in chunks due to API limits
    for i in range(0, length, max_query_length):
        api_string = ''
        if i+max_query_length > length:
            y = length - i
        else:
            y = max_query_length
        #format string of lat's and long's to include in api request to Mapquest
        hold_df = location_df.iloc[i:i+y]
        hold_df = hold_df.reset_index(drop=True)
        #print(hold_df)
        for index, row in hold_df.iterrows():
            if index == 0:
                api_string = api_string+str(row[2])+','+str(row[3])
            else:
                api_string = api_string+','+str(row[2])+','+str(row[3])
        print(api_string)
        response = requests.get('http://open.mapquestapi.com/elevation/v1/profile?key='+API_KEY+'&shapeFormat=raw&unit=f&latLngCollection='+api_string)
        #parse json for elevations
        elev = json.loads(response.text)
        print(elev)
        #print(y)
        for index, x in enumerate(list(range(i,i+y))):
            #print(index, x)
            #print(elev['elevationProfile'][index]['height'])
            if index in range(len(elev['elevationProfile'])):
                elevation = elev['elevationProfile'][index]['height']
                location_df.iloc[x,4] = elevation
            else:
                location_df.iloc[x,4] = -32768
    location_df.sort_index(inplace=True)
    return location_df

## Example function call

In [ ]:
#df data = whole dataset
data.drop(data[data['LAT_016'] == 0].index,inplace=True)
elevations = geteachelevation(data)
elevations.to_csv('elevations.csv')